In [1]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

top_k = 10

model_name = '/mnt/workspace/.cache/modelscope/hub/maple77/zpoint_large_embedding_zh'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = Chroma(persist_directory="soybean_db2", embedding_function=hf)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": top_k}
)

/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
question = "高产量的大豆在生长过程中有什么特性？"
retri_re = retriever.invoke(question)

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/mnt/workspace/.cache/modelscope/hub/Xorbits/bge-reranker-base')
rerank_model = AutoModelForSequenceClassification.from_pretrained('/mnt/workspace/.cache/modelscope/hub/Xorbits/bge-reranker-base')

In [ ]:
import torch
pairs = []
for idx in range(len(retri_re)):
    pairs.append([question, retri_re[idx].page_content])

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
combined = sorted(zip(scores, retri_re), reverse=True)
scores_rerank_list, retri_rerank_list = zip(*combined)
# print(scores_rerank_list, retri_rerank_list)
retri_rerank_list

In [14]:
from langchain import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
template = """
You are an expert in phenomics in agronomy, and you have a very rich knowledge of agronomy and phenomics.
Use the context snippets retrieved below to answer the agronomy question from a phenotypic point of view.
If if you don't know the answer, say you don't know.

Question: {question} 

Context: {context} 

Answer:
"""
llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4-0520",
    openai_api_key="661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = PromptTemplate(
    template=template, 
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] template="\nYou are an expert in phenomics in agronomy, and you have a very rich knowledge of agronomy and phenomics.\nUse the context snippets retrieved below to answer the agronomy question from a phenotypic point of view.\nIf if you don't know the answer, say you don't know.\n\nQuestion: {question} \n\nContext: {context} \n\nAnswer:\n"


In [25]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
# memory = ConversationBufferMemory(k = 5, memory_key="chat_history", return_messages=True)
# llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
rag_chain = (
    {"context": RunnablePassthrough(),  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

In [16]:
context_tmp = ''.join(qr.page_content for qr in retri_rerank_list)
inputs = {"context": context_tmp, "question": question}
ans = rag_chain.invoke(inputs)
print(ans)

高产量的大豆在生长过程中的特性主要包括：

1. **生长速率（R1-R5阶段）**：从生殖生长的初期（R1，即任意节位出现一朵花）到豆荚开始发育（R5，即当豆荚被挤压时可以感觉到豆子，在最高的四个节位之一有完全展开的叶子）这一阶段的作物生长速率是决定每单位面积种子数量的主要因素，因此也是与产量最相关的组成部分。

2. **对胁迫的敏感性**：大豆在生殖生长阶段对干旱非常敏感。如果开花期间发生干旱，大豆会产生较少的花、豆荚和种子，主要影响豆荚的数量，而不是种子大小。

3. **生物量分配**：R1-R5阶段的生物量分配对最终产量有重要影响。

4. **有效填充期（EFP）**：种子生长速率（即种子干物质每日增加量）和最终种子大小（在R8，即完全成熟时测量）是决定产量的关键因素。较长的有效填充期通常与达到最大种子大小有关。

5. **其他农艺性状**：如冠层或根系结构、耐淹、耐寒、耐旱性以及种子质量等性状也会影响最终产量，但这些性状的影响通常是通过影响上述产量相关组分而体现的。

综上所述，高产量大豆的主要特性是在关键生长阶段（尤其是R1-R5）具有高的生长速率，以及对环境胁迫有较好的耐受性或适应性。


In [41]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
system_prompt = """You are an expert in phenomics in agronomy, and you have a very rich knowledge of agronomy and phenomics.
Use the context snippets retrieved below to answer the agronomy question from a phenotypic point of view.Please try to answer questions in terms of phenotype, such as growth rate, vegetation index, biomass, etc
If if you don't know the answer, say you don't know.The following is the reference document:\n\n{context}"""
chat_history = []
qa_prompt = ChatPromptTemplate.from_messages(
[
("system", system_prompt),
        MessagesPlaceholder("chat_history"),
("human", "Question: {question} Answer:"),
]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [42]:
from langchain_core.messages import AIMessage, HumanMessage
ans = question_answer_chain.invoke({"context": retri_rerank_list, "question": question, "chat_history": chat_history})
print(ans)
chat_history.extend(
[
    HumanMessage(content=question),
    AIMessage(content=ans),
]
)

高产量大豆在生长过程中的特性可以从以下几个方面进行描述：

1. **生长速率（Crop Growth Rate R1-R5）**：特别是从R1（首次开花）到R5（豆荚中的豆子开始发育）这一阶段，是决定单位面积种子数量的主要因素，因此这一阶段的生长速率是影响产量的最重要组成部分。较高的干物质积累率（以克干物质/单位土地面积/单位时间来衡量）是高产量大豆的重要特征。

2. **生物量分配**：在R1-R5期间，植物生物量的分配对最终产量有显著影响。理想的分配模式应最大程度地支持种子的形成和发育。

3. **对逆境的耐受性**：大豆对生殖生长阶段的干旱非常敏感。高产量的大豆品种往往表现出更好的干旱耐受性，减少因干旱导致的开花、结荚和种子数量的减少。

4. **有效填充期（EFP）**：高产量大豆品种通常有较长的有效填充期，即种子线性干物质积累期较长，这通常与达到最大种子大小有关。

5. **叶面积和叶面积持续时间**：较大的叶面积和较长的叶面积持续时间通常有助于提高光合同化效率，从而对生物量和产量产生积极影响。

6. **冠层和根系结构**：理想的冠层结构和根系分布有助于最大化对光、水和营养资源的利用，进而影响产量相关的表型特征。

7. **与产量直接相关的其他表型特征**：例如，对洪涝、寒冷的耐受性以及种子质量等，虽然这些特征对产量的影响可能通过影响R1-R5期间的生物量积累和分配来实现，但它们本身也是高产量大豆的重要表型特征。

综上，高产量大豆在生长过程中的特性包括快速的生长速率、有效的生物量分配、对逆境的耐受性、较长的有效填充期、理想的冠层和根系结构以及良好的种子质量等。


In [44]:
ans = question_answer_chain.invoke({"context": retri_rerank_list, "question": '生长速率是什么', "chat_history": chat_history})

In [47]:
ans

'生长速率是指植物在单位时间内干物质的增加量，通常用来衡量植物生长的快慢。在农业生产和研究中，生长速率是一个重要的表型特征，它可以用克干物质积累每单位土地面积每单位时间（例如，每天）来表示。在特定的大豆生长阶段，例如从R1（首次开花）到R5（豆子开始发育）阶段，生长速率是决定种子数量和最终产量的关键因素。高的生长速率意味着植物在这个阶段能够快速积累干物质，有利于形成更多的种子和提高产量。'